In [ ]:
# written by Maximilian Ullherr, maximilian.ullherr@physik.uni-wuerzburg.de,
# Lehrstuhl fuer Roentgenmikroskopie/Universitaet Wuerzburg, Josef-Martin-Weg 63, 97074 Wuerzburg, Germany
# last updated 2019-02-07

%pylab
import sys; sys.path.append('..')  # copy the pyETX module either into the folder with the notebook and remove this line
from MTF_evaluation import *  # only required import, will import two functions
from notebook import *
print('kernel ready')

print('\n=== printing the docstring for the main evaluation function ===')
help(MTF_Evaluator)

## Simulation 

shows how this works and show example evaluation

    a) generate test data
    
    b) estimate MTF with edge angle not known
    
    c) example for inaccurate direct MTF estimate

In [ ]:
# a) generate test data
from MTF_evaluation import _bin_avg_1d, MTF_gauss
from scipy.ndimage import gaussian_filter

smooth_strength = 1.5  # MTF evaluation will work for values between 0.25 and 25
rep_len = 10
intensity = 1000
dark = 80

edge_image = zeros((200, 150), 'f4')
for k in range(edge_image.shape[0]):
    edge = zeros(rep_len*edge_image.shape[1], 'f4')
    start = edge.shape[0]//2-edge_image.shape[0]//2
    edge[:start+k] = 1.
    edge = _bin_avg_1d(edge, rep_len)
    edge_image[k, :] = edge

edge_image = gaussian_filter(edge_image, smooth_strength)
print(rad2deg(arctan(1/rep_len)))

edge_image = (random.poisson(edge_image*intensity+dark)/(intensity+dark)).astype('f4')
    
plt.clf()
plt.imshow(edge_image)
plt.title('simulated edge image')
display(plt.gcf())
plt.close('all')

In [ ]:
# b) evaluate MTF and estimate the edge angle automatically
MTFE = MTF_Evaluator(edge_image, 'test', edge_angle=-rad2deg(arctan(1/rep_len))+1e-4)
print(f'correct sigma: {smooth_strength:.5g}')

print('fit parameters covariance matrix:\n', MTFE.fit_pcov)  # additional results are available

if MTFE.fig is not None:
    MTFE.fig.axes[2].plot(MTFE.u, MTF_gauss(MTFE.u, smooth_strength), label='actual MTF')
    MTFE.fig.axes[2].legend()
    display(MTFE.fig)

In [ ]:
# c) simple fourier MTF evaluation, does not work accurately at higher frequences (no noise treatment)
from MTF_evaluation import get_MTF_ft
MTF_coords, MTF, fig = get_MTF_ft(edge_image, None, plot_figure=True)

if fig is not None:
    fig.axes[0].plot(MTF_coords, MTF_gauss(MTF_coords, smooth_strength), label='actual MTF')
    fig.axes[0].legend()
    display(fig)
    print('this algorithm is known to not work accurately on noisy data, use the one above')

In [ ]:
plt.close('all')